**CRIANDO UMA TABELA DE LOG**

In [1]:
USE [DW_09]
GO
IF NOT EXISTS
  (SELECT * FROM sysobjects WHERE name='LOG_CARGAS' AND xtype='U')

CREATE TABLE LOG_CARGAS
 (ID INT IDENTITY(1,1) NOT NULL PRIMARY KEY,
 NUMERO_ERRO INT,
 SERVERIDADE_ERRO INT,
 ESTADO_ERRO  INT,
 PROC_ERRO NVARCHAR(255),
 LINHA_ERRO INT,
 MSG_ERRO NVARCHAR(255),
 SITUACAO NVARCHAR(10),
 PROCESSO NVARCHAR(30),
 DATA_INI DATETIME,
 DATA_FIM DATETIME  
 )

Commands completed successfully.

Commands completed successfully.

Total execution time: 00:00:00.019

**CRIANDO TABELA STG VENDAS**

In [2]:
USE [DW_09]
GO
IF NOT EXISTS
  (SELECT * FROM sysobjects WHERE name='STG_VENDAS' AND xtype='U')
CREATE TABLE [dbo].[STG_VENDAS](
	[SEGMENTO] [varchar](255) NULL,
	[PAIS] [varchar](255) NULL,
	[PRODUTO] [varchar](255) NULL,
	[TIPO_DESCONTO] [varchar](255) NULL,
	[QTDE_VENDA] [varchar](255) NULL,
	[PRECO_CUSTO] [varchar](255) NULL,
	[DATA_VENDA] [date] NULL,
	[PECO_VENDA] [varchar](255) NULL,
	[VENDA_BRUTA] [varchar](255) NULL,
	[DESCONTO] [varchar](255) NULL,
	[LUCRO] [varchar](255) NULL,
	[VENDAS] [varchar](255) NULL,
	[CUSTO_VENDA] [varchar](255) NULL,
) ON [PRIMARY]
GO



Commands completed successfully.

Commands completed successfully.

Total execution time: 00:00:00.007

**TRUNCANDO OS DADOS DA TABELA**

In [3]:
TRUNCATE TABLE STG_VENDAS;

Commands completed successfully.

Total execution time: 00:00:00.002

**CARREGANDO DADOS NA TABELA STAGIN COM BULK INSERT**

In [4]:
USE [DW_09]
GO
BEGIN TRANSACTION;  
	DECLARE @VDATA_INI DATETIME;
	SET @VDATA_INI= GETDATE();

BEGIN TRY  

BULK INSERT STG_VENDAS
FROM 'C:\DESAFIO\BaseDadosDesafio.csv'
WITH        (
	codepage='650001',  -- { 'ACP' | 'OEM' | 'RAW' | 'code_page' } ] 65001
    DATAFILETYPE = 'char',   --      { 'char' | 'native'| 'widechar' | 'widenative' } ]         
	fieldterminator=';',               
	rowterminator='\n',               
	maxerrors = 0,               
	fire_triggers,             
    firstrow = 2               
   --lastrow = 10
	     ) 
END TRY  
BEGIN CATCH
 	IF @@TRANCOUNT > 0  
        ROLLBACK TRANSACTION; 

INSERT INTO [dbo].[LOG_CARGAS]
           ([NUMERO_ERRO],
           [SERVERIDADE_ERRO],
           [ESTADO_ERRO],
           [PROC_ERRO],
           [LINHA_ERRO],
           [MSG_ERRO],
           [SITUACAO],
           [PROCESSO],
           [DATA_INI],
		   [DATA_FIM])
     VALUES
        (ERROR_NUMBER(),
        ERROR_SEVERITY(),
        ERROR_STATE(),
        ERROR_PROCEDURE(),
        ERROR_LINE() ,
        ERROR_MESSAGE() ,
        'ERRO',
        'CARGA BULK STG',
		@VDATA_INI,
		GETDATE()
		)

        
END CATCH;  

INSERT INTO LOG_CARGAS
           ([NUMERO_ERRO],
           [SERVERIDADE_ERRO],
           [ESTADO_ERRO],
           [PROC_ERRO],
           [LINHA_ERRO],
           [MSG_ERRO],
           [SITUACAO],
           [PROCESSO],
           [DATA_INI],
		   [DATA_FIM])
     VALUES
        (ERROR_NUMBER(),
        ERROR_SEVERITY(),
        ERROR_STATE(),
        ERROR_PROCEDURE(),
        ERROR_LINE() ,
        ERROR_MESSAGE() ,
        'SUCESSO',
        'CARGA BULK STG',
		@VDATA_INI,
		GETDATE()
		)

IF @@TRANCOUNT > 0  
    COMMIT TRANSACTION;  
GO  



Commands completed successfully.

(700 rows affected)

(1 row affected)

Total execution time: 00:00:00.080

**CRIANDO TABELAS DIMENSÕES**

In [5]:
USE [DW_09]
GO
--CRIANDO DIMENSAO SEGMENTO
IF NOT EXISTS
  (SELECT * FROM sysobjects WHERE name='D_SEGMENTO' AND xtype='U')
CREATE TABLE [dbo].[D_SEGMENTO](
	[Id_SEGMENTO] [int] IDENTITY(1,1) NOT NULL,
	[NOME_SEGMENTO] [varchar](255) NULL,
PRIMARY KEY CLUSTERED 
(
	[Id_SEGMENTO] ASC
)WITH (PAD_INDEX = OFF, STATISTICS_NORECOMPUTE = OFF, IGNORE_DUP_KEY = OFF, ALLOW_ROW_LOCKS = ON, ALLOW_PAGE_LOCKS = ON, OPTIMIZE_FOR_SEQUENTIAL_KEY = OFF) ON [PRIMARY]
) ON [PRIMARY]
GO

--CRIANDO DIMENSAO PAIS
IF NOT EXISTS
  (SELECT * FROM sysobjects WHERE name='D_PAIS' AND xtype='U')
CREATE TABLE [dbo].[D_PAIS](
	[Id_PAIS] [int] IDENTITY(1,1) NOT NULL,
	[NOME_PAIS] [varchar](255) NULL,
PRIMARY KEY CLUSTERED 
(
	[Id_PAIS] ASC
)WITH (PAD_INDEX = OFF, STATISTICS_NORECOMPUTE = OFF, IGNORE_DUP_KEY = OFF, ALLOW_ROW_LOCKS = ON, ALLOW_PAGE_LOCKS = ON, OPTIMIZE_FOR_SEQUENTIAL_KEY = OFF) ON [PRIMARY]
) ON [PRIMARY]
GO
--CRIANDO DIMENSAO PRODUTO
IF NOT EXISTS
  (SELECT * FROM sysobjects WHERE name='D_PRODUTO' AND xtype='U')

CREATE TABLE [dbo].[D_PRODUTO](
	[Id_PRODUTO] [int] IDENTITY(1,1) NOT NULL,
	[NOME_PRODUTO] [varchar](255) NULL,
PRIMARY KEY CLUSTERED 
(
	[Id_PRODUTO] ASC
)WITH (PAD_INDEX = OFF, STATISTICS_NORECOMPUTE = OFF, IGNORE_DUP_KEY = OFF, ALLOW_ROW_LOCKS = ON, ALLOW_PAGE_LOCKS = ON, OPTIMIZE_FOR_SEQUENTIAL_KEY = OFF) ON [PRIMARY]
) ON [PRIMARY]
GO

--CRIANDO DIMENSAO TIPO_DESCONTO
IF NOT EXISTS
  (SELECT * FROM sysobjects WHERE name='D_TIPO_DESCONTO' AND xtype='U')

CREATE TABLE [dbo].[D_TIPO_DESCONTO](
	[Id_TIPO_DESCONTO] [int] IDENTITY(1,1) NOT NULL,
	[NOME_TIPO_DESCONTO] [varchar](255) NULL,
PRIMARY KEY CLUSTERED 
(
	[Id_TIPO_DESCONTO] ASC
)WITH (PAD_INDEX = OFF, STATISTICS_NORECOMPUTE = OFF, IGNORE_DUP_KEY = OFF, ALLOW_ROW_LOCKS = ON, ALLOW_PAGE_LOCKS = ON, OPTIMIZE_FOR_SEQUENTIAL_KEY = OFF) ON [PRIMARY]
) ON [PRIMARY]
GO

--CRIANDO DIMENSAO DATA_VENDA
IF NOT EXISTS
  (SELECT * FROM sysobjects WHERE name='D_DATA_VENDA' AND xtype='U')
CREATE TABLE [dbo].[D_DATA_VENDA](
	[Id_DATA_VENDA] [int] IDENTITY(1,1) NOT NULL,
	[NOME_DATA_VENDA] [date] NULL,
PRIMARY KEY CLUSTERED 
(
	[Id_DATA_VENDA] ASC
)WITH (PAD_INDEX = OFF, STATISTICS_NORECOMPUTE = OFF, IGNORE_DUP_KEY = OFF, ALLOW_ROW_LOCKS = ON, ALLOW_PAGE_LOCKS = ON, OPTIMIZE_FOR_SEQUENTIAL_KEY = OFF) ON [PRIMARY]
) ON [PRIMARY]
GO

Commands completed successfully.

Commands completed successfully.

Commands completed successfully.

Commands completed successfully.

Commands completed successfully.

Commands completed successfully.

Total execution time: 00:00:00.068

**CARGA DIMENSÃO SEGMENTO**

In [6]:
BEGIN TRANSACTION;  
	DECLARE @VDATA_INI DATETIME;
	SET @VDATA_INI= GETDATE();
BEGIN TRY  

MERGE D_SEGMENTO AS Destino

USING (SELECT DISTINCT SEGMENTO FROM STG_VENDAS) AS Origem

ON Destino.NOME_SEGMENTO = Origem.SEGMENTO

-- Há registro no destino e na origem
WHEN MATCHED 

THEN 
    UPDATE SET NOME_SEGMENTO = Origem.SEGMENTO

--Quando não há registro no destino e há na origem
WHEN NOT MATCHED 

THEN 
    INSERT (NOME_SEGMENTO) VALUES (Origem.SEGMENTO);

   END TRY  
BEGIN CATCH
 	IF @@TRANCOUNT > 0  
        ROLLBACK TRANSACTION; 

INSERT INTO [dbo].[LOG_CARGAS]
           ([NUMERO_ERRO],
           [SERVERIDADE_ERRO],
           [ESTADO_ERRO],
           [PROC_ERRO],
           [LINHA_ERRO],
           [MSG_ERRO],
           [SITUACAO],
           [PROCESSO],
           [DATA_INI],
		   [DATA_FIM])
     VALUES
        (ERROR_NUMBER(),
        ERROR_SEVERITY(),
        ERROR_STATE(),
        ERROR_PROCEDURE(),
        ERROR_LINE() ,
        ERROR_MESSAGE() ,
        'ERRO',
        'CARGA SEGMENTO',
		@VDATA_INI,
		GETDATE()
		)

        
END CATCH;  

INSERT INTO LOG_CARGAS
           ([NUMERO_ERRO],
           [SERVERIDADE_ERRO],
           [ESTADO_ERRO],
           [PROC_ERRO],
           [LINHA_ERRO],
           [MSG_ERRO],
           [SITUACAO],
           [PROCESSO],
           [DATA_INI],
		   [DATA_FIM])
     VALUES
        (ERROR_NUMBER(),
        ERROR_SEVERITY(),
        ERROR_STATE(),
        ERROR_PROCEDURE(),
        ERROR_LINE() ,
        ERROR_MESSAGE() ,
        'SUCESSO',
       'CARGA SEGMENTO',
		@VDATA_INI,
		GETDATE()
		)

IF @@TRANCOUNT > 0  
    COMMIT TRANSACTION;  
GO  

 


(5 rows affected)

(1 row affected)

Total execution time: 00:00:00.031

**CARGA DIMENSAO PAIS**

In [13]:
BEGIN TRANSACTION;  
	DECLARE @VDATA_INI DATETIME;
	SET @VDATA_INI= GETDATE();
BEGIN TRY  


MERGE D_PAIS AS Destino

USING (SELECT DISTINCT PAIS FROM STG_VENDAS) AS Origem

ON Destino.NOME_PAIS= Origem.PAIS

-- Há registro no destino e na origem
WHEN MATCHED 

THEN 
    UPDATE SET NOME_PAIS = Origem.PAIS

--Quando não há registro no destino e há na origem
WHEN NOT MATCHED 

THEN 
    INSERT (NOME_PAIS) VALUES (Origem.PAIS);

END TRY  
BEGIN CATCH
 	IF @@TRANCOUNT > 0  
        ROLLBACK TRANSACTION; 

INSERT INTO [dbo].[LOG_CARGAS]
           ([NUMERO_ERRO],
           [SERVERIDADE_ERRO],
           [ESTADO_ERRO],
           [PROC_ERRO],
           [LINHA_ERRO],
           [MSG_ERRO],
           [SITUACAO],
           [PROCESSO],
           [DATA_INI],
		   [DATA_FIM])
     VALUES
        (ERROR_NUMBER(),
        ERROR_SEVERITY(),
        ERROR_STATE(),
        ERROR_PROCEDURE(),
        ERROR_LINE() ,
        ERROR_MESSAGE() ,
        'ERRO',
        'CARGA PAIS',
		@VDATA_INI,
		GETDATE()
		)

        
END CATCH;  

INSERT INTO LOG_CARGAS
           ([NUMERO_ERRO],
           [SERVERIDADE_ERRO],
           [ESTADO_ERRO],
           [PROC_ERRO],
           [LINHA_ERRO],
           [MSG_ERRO],
           [SITUACAO],
           [PROCESSO],
           [DATA_INI],
		   [DATA_FIM])
     VALUES
        (ERROR_NUMBER(),
        ERROR_SEVERITY(),
        ERROR_STATE(),
        ERROR_PROCEDURE(),
        ERROR_LINE() ,
        ERROR_MESSAGE() ,
        'SUCESSO',
        'CARGA PAIS',
		@VDATA_INI,
		GETDATE()
		)

IF @@TRANCOUNT > 0  
    COMMIT TRANSACTION;  
GO  



(5 rows affected)

(1 row affected)

Total execution time: 00:00:00.023

**CARGA DIMENSÃO TIPO\_DESCONTO**

In [8]:
BEGIN TRANSACTION;  
	DECLARE @VDATA_INI DATETIME;
    DECLARE @VPROCESSO VARCHAR(30);
	SET @VDATA_INI= GETDATE();
    SET @VPROCESSO= 'CARGA TIPO_DESCONTO'
BEGIN TRY  


MERGE D_TIPO_DESCONTO AS Destino

USING (SELECT DISTINCT TIPO_DESCONTO FROM STG_VENDAS) AS Origem

ON Destino.NOME_TIPO_DESCONTO = Origem.TIPO_DESCONTO

-- Há registro no destino e na origem
WHEN MATCHED 

THEN 
    UPDATE SET NOME_TIPO_DESCONTO= Origem.TIPO_DESCONTO

--Quando não há registro no destino e há na origem
WHEN NOT MATCHED 

THEN 
    INSERT (NOME_TIPO_DESCONTO) VALUES (Origem.TIPO_DESCONTO);

 END TRY  
BEGIN CATCH
 	IF @@TRANCOUNT > 0  
        ROLLBACK TRANSACTION; 

INSERT INTO [dbo].[LOG_CARGAS]
           ([NUMERO_ERRO],
           [SERVERIDADE_ERRO],
           [ESTADO_ERRO],
           [PROC_ERRO],
           [LINHA_ERRO],
           [MSG_ERRO],
           [SITUACAO],
           [PROCESSO],
           [DATA_INI],
		   [DATA_FIM])
     VALUES
        (ERROR_NUMBER(),
        ERROR_SEVERITY(),
        ERROR_STATE(),
        ERROR_PROCEDURE(),
        ERROR_LINE() ,
        ERROR_MESSAGE() ,
        'ERRO',
        @VPROCESSO,
		@VDATA_INI,
		GETDATE()
		)

        
END CATCH;  

INSERT INTO LOG_CARGAS
           ([NUMERO_ERRO],
           [SERVERIDADE_ERRO],
           [ESTADO_ERRO],
           [PROC_ERRO],
           [LINHA_ERRO],
           [MSG_ERRO],
           [SITUACAO],
           [PROCESSO],
           [DATA_INI],
		   [DATA_FIM])
     VALUES
        (ERROR_NUMBER(),
        ERROR_SEVERITY(),
        ERROR_STATE(),
        ERROR_PROCEDURE(),
        ERROR_LINE() ,
        ERROR_MESSAGE() ,
        'SUCESSO',
        @VPROCESSO,
		@VDATA_INI,
		GETDATE()
		)

IF @@TRANCOUNT > 0  
    COMMIT TRANSACTION;  
GO  

   


(4 rows affected)

(1 row affected)

Total execution time: 00:00:00.021

**CARGA DIMENSÃO PRODUTO**

In [14]:
BEGIN TRANSACTION;  
	DECLARE @VDATA_INI DATETIME;
    DECLARE @VPROCESSO VARCHAR(30);
	SET @VDATA_INI= GETDATE();
    SET @VPROCESSO= 'CARGA PRODUTO'
BEGIN TRY  


MERGE D_PRODUTO AS Destino

USING (SELECT DISTINCT PRODUTO FROM STG_VENDAS) AS Origem

ON Destino.NOME_PRODUTO = Origem.PRODUTO

-- Há registro no destino e na origem
WHEN MATCHED 

THEN 
    UPDATE SET NOME_PRODUTO= Origem.PRODUTO

--Quando não há registro no destino e há na origem
WHEN NOT MATCHED 

THEN 
    INSERT (NOME_PRODUTO) VALUES (Origem.PRODUTO);

 END TRY  
BEGIN CATCH
 	IF @@TRANCOUNT > 0  
        ROLLBACK TRANSACTION; 

INSERT INTO [dbo].[LOG_CARGAS]
           ([NUMERO_ERRO],
           [SERVERIDADE_ERRO],
           [ESTADO_ERRO],
           [PROC_ERRO],
           [LINHA_ERRO],
           [MSG_ERRO],
           [SITUACAO],
           [PROCESSO],
           [DATA_INI],
		   [DATA_FIM])
     VALUES
        (ERROR_NUMBER(),
        ERROR_SEVERITY(),
        ERROR_STATE(),
        ERROR_PROCEDURE(),
        ERROR_LINE() ,
        ERROR_MESSAGE() ,
        'ERRO',
        @VPROCESSO,
		@VDATA_INI,
		GETDATE()
		)
END CATCH;  

INSERT INTO LOG_CARGAS
           ([NUMERO_ERRO],
           [SERVERIDADE_ERRO],
           [ESTADO_ERRO],
           [PROC_ERRO],
           [LINHA_ERRO],
           [MSG_ERRO],
           [SITUACAO],
           [PROCESSO],
           [DATA_INI],
		   [DATA_FIM])
     VALUES
        (ERROR_NUMBER(),
        ERROR_SEVERITY(),
        ERROR_STATE(),
        ERROR_PROCEDURE(),
        ERROR_LINE() ,
        ERROR_MESSAGE() ,
        'SUCESSO',
        @VPROCESSO,
		@VDATA_INI,
		GETDATE()
		)

IF @@TRANCOUNT > 0  
    COMMIT TRANSACTION;  
GO  


(6 rows affected)

(1 row affected)

Total execution time: 00:00:00.021

**CARGA DIMENSÃO DATA DA VENDA**

In [10]:
BEGIN TRANSACTION;  
	DECLARE @VDATA_INI DATETIME;
    DECLARE @VPROCESSO VARCHAR(30);
	SET @VDATA_INI= GETDATE();
    SET @VPROCESSO= 'CARGA DATA_VENDA'
BEGIN TRY  


MERGE D_DATA_VENDA AS Destino

USING (SELECT DISTINCT DATA_VENDA FROM STG_VENDAS) AS Origem

ON Destino.NOME_DATA_VENDA = Origem.DATA_VENDA

-- Há registro no destino e na origem
WHEN MATCHED 

THEN 
    UPDATE SET NOME_DATA_VENDA= Origem.DATA_VENDA

--Quando não há registro no destino e há na origem
WHEN NOT MATCHED 

THEN 
    INSERT (NOME_DATA_VENDA) VALUES (Origem.DATA_VENDA);

 END TRY 

 BEGIN CATCH
 	IF @@TRANCOUNT > 0  
        ROLLBACK TRANSACTION; 

INSERT INTO [dbo].[LOG_CARGAS]
           ([NUMERO_ERRO],
           [SERVERIDADE_ERRO],
           [ESTADO_ERRO],
           [PROC_ERRO],
           [LINHA_ERRO],
           [MSG_ERRO],
           [SITUACAO],
           [PROCESSO],
           [DATA_INI],
		   [DATA_FIM])
     VALUES
        (ERROR_NUMBER(),
        ERROR_SEVERITY(),
        ERROR_STATE(),
        ERROR_PROCEDURE(),
        ERROR_LINE() ,
        ERROR_MESSAGE() ,
        'ERRO',
        @VPROCESSO,
		@VDATA_INI,
		GETDATE()
		)
END CATCH;  
INSERT INTO LOG_CARGAS
           ([NUMERO_ERRO],
           [SERVERIDADE_ERRO],
           [ESTADO_ERRO],
           [PROC_ERRO],
           [LINHA_ERRO],
           [MSG_ERRO],
           [SITUACAO],
           [PROCESSO],
           [DATA_INI],
		   [DATA_FIM])
     VALUES
        (ERROR_NUMBER(),
        ERROR_SEVERITY(),
        ERROR_STATE(),
        ERROR_PROCEDURE(),
        ERROR_LINE() ,
        ERROR_MESSAGE() ,
        'SUCESSO',
        @VPROCESSO,
		@VDATA_INI,
		GETDATE()
		)

IF @@TRANCOUNT > 0  
    COMMIT TRANSACTION;  
GO  


(1 row affected)

(1 row affected)

Total execution time: 00:00:00.018

**CRIANDO A TABELA FATO**

In [11]:
IF NOT EXISTS
  (SELECT * FROM sysobjects WHERE name='FATO_VENDA' AND xtype='U')
CREATE TABLE FATO_VENDA
(
Id_SEGMENTO [int] NULL,
Id_PAIS  [int] NULL,
Id_PRODUTO  [int] NULL,
Id_TIPO_DESCONTO  [int] NULL,
QTDE_VENDA  DECIMAL(10,2) NULL,
PRECO_CUSTO   DECIMAL(10,2) NULL,
PRECO_VENDA  DECIMAL(10,2) NULL,
VENDA_BRUTA   DECIMAL(10,2) NULL,
DESCONTO  DECIMAL(10,2) NULL,
VENDAS  DECIMAL(10,2) NULL,
CUSTO_VENDAS DECIMAL(10,2) NULL,
LUCRO DECIMAL(10,2) NULL,
ID_DATA_VENDA DATE,
--Id_VENDA  [int] NOT NULL PRIMARY KEY,
)

Commands completed successfully.

Total execution time: 00:00:00.009

**ANALISANDO AMOSTRAGEM DOS DADOS**

In [21]:
SELECT b.Id_SEGMENTO,
       c.Id_PAIS,
       d.Id_PRODUTO,
       e.Id_TIPO_DESCONTO,
       f.Id_DATA_VENDA,
       a.QTDE_VENDA,
       a.PRECO_CUSTO,
       a.PECO_VENDA,
       a.VENDA_BRUTA,
       a.DESCONTO,
       a.VENDAS,
       a.CUSTO_VENDA,
       a.LUCRO    
  FROM STG_VENDAS a
   INNER JOIN D_SEGMENTO b
	ON a.SEGMENTO=b.NOME_SEGMENTO

   INNER JOIN D_PAIS c
	ON a.PAIS=c.NOME_PAIS

  INNER JOIN D_PRODUTO d
	ON a.PRODUTO=d.NOME_PRODUTO

  INNER JOIN D_TIPO_DESCONTO e
	ON a.TIPO_DESCONTO=e.NOME_TIPO_DESCONTO

  INNER JOIN D_DATA_VENDA f
	ON a.DATA_VENDA=f.NOME_DATA_VENDA

  

(2100 rows affected)

Total execution time: 00:00:00.131

Id_SEGMENTO,Id_PAIS,Id_PRODUTO,Id_TIPO_DESCONTO,Id_DATA_VENDA,QTDE_VENDA,PRECO_CUSTO,PECO_VENDA,VENDA_BRUTA,DESCONTO,VENDAS,CUSTO_VENDA,LUCRO
3,3,2,4,1,1513,3,529550,0,529550,136170,01/12/2014,393380
3,3,4,4,1,1006,10,352100,0,352100,90540,01/06/2014,261560
3,1,4,4,1,1725,10,603750,0,603750,155250,01/11/2013,448500
3,3,4,4,1,1513,10,529550,0,529550,136170,01/12/2014,393380
3,3,5,4,1,1006,120,352100,0,352100,90540,01/06/2014,261560
3,2,6,4,1,1527,250,534450,0,534450,137430,01/09/2013,397020
3,2,1,4,1,2750,260,962500,0,962500,247500,01/02/2014,715000
3,4,2,2,1,1210,3,423500,4235,419265,104665,01/03/2014,314600
3,4,2,2,1,1397,3,488950,"4889,5","484060,5","120840,5",01/10/2014,363220
3,2,2,2,1,2155,3,754250,"7542,5","746707,5","186407,5",01/12/2014,560300


**AJUSTE LEFT JOIN E CONVERT**

In [15]:
BEGIN TRANSACTION;  
   DECLARE @VDATA_INI DATETIME;
    DECLARE @VPROCESSO VARCHAR(30);
    SET @VDATA_INI= GETDATE();
    SET @VPROCESSO= 'CARGA FATO'
BEGIN TRY  


MERGE FATO_VENDA AS Destino

USING (
SELECT 
       b.Id_SEGMENTO,
       c.Id_PAIS,
       d.Id_PRODUTO,
       e.Id_TIPO_DESCONTO,
       f.Id_DATA_VENDA,
       COALESCE(g.id_Pais,999) as Id_PAIS,
       CAST(REPLACE(a.QTDE_VENDA,',','.') AS decimal(10,2)) AS QTDE_VENDA,
       CAST(REPLACE(a.PRECO_CUSTO,',','.') AS decimal(10,2))AS PRECO_CUSTO,
       CAST(REPLACE(a.PECO_VENDA,',','.') AS decimal(10,2))AS PECO_VENDA,
       CAST(REPLACE(a.VENDA_BRUTA,',','.') AS decimal(10,2))AS VENDA_BRUTA,
       CAST(REPLACE(a.VENDA_BRUTA,',','.') AS decimal(10,2))AS VENDA_BRUTA,
       CAST(REPLACE(a.DESCONTO,',','.') AS decimal(10,2))AS DESCONTO,
       CAST(REPLACE(a.VENDAS,',','.') AS decimal(10,2))AS VENDAS,
       CAST(REPLACE(a.CUSTO_VENDA,',','.') AS decimal(10,2))AS CUSTO_VENDA,
       CAST(REPLACE(a.LUCRO,',','.') AS decimal(10,2))AS LUCRO
  FROM STG_VENDAS a
    LEFT JOIN D_SEGMENTO b
	  ON a.SEGMENTO=b.NOME_SEGMENTO

    LEFT JOIN D_PAIS c
	  ON a.PAIS=c.NOME_PAIS

    LEFT JOIN D_PRODUTO d
	  ON a.PRODUTO=d.NOME_PRODUTO

    LEFT JOIN D_TIPO_DESCONTO e
	  ON a.TIPO_DESCONTO=e.NOME_TIPO_DESCONTO

    LEFT JOIN D_DATA_VENDA f
	  ON a.DATA_VENDA=f.NOME_DATA_VENDA) AS Origem

 ON Destino.VENDAS = Origem.VENDAS
WHEN MATCHED 
 THEN 
    UPDATE SET  Id_SEGMENTO=origem.Id_SEGMENTO, 
                Id_PAIS=origem.Id_PAIS,
                Id_PRODUTO=origem.Id_PRODUTO,
                Id_TIPO_DESCONTO=origem.Id.TIPO_DESCONTO,
                QTDE_VENDA=origem.QTDE_VENDA,
                PRECO_CUSTO=origem.PRECO_CUSTO,
                PECO_VENDA=origem.PECO_VENDA,
                VENDA_BRUTA=origem.VENDA_BRUTA,
                DESCONTO=origem.DESCONTO,
                VENDAS=origem.VENDAS,
                CUSTO_VENDA=origem_CUSTO_VENDA,
                LUCRO=origem.LUCRO

--Quando não há registro no destino e há na origem
WHEN NOT MATCHED 
 THEN 
   INSERT  
      (Id_SEGMENTO,
       Id_PAIS,
       Id_PRODUTO,
       Id_TIPO_DESCONTO,
       Id_DATA_VENDA,
       QTDE_VENDA,
       PRECO_CUSTO,
       PECO_VENDA,
       VENDA_BRUTA,
       DESCONTO,
       VENDAS,
       CUSTO_VENDA,
       LUCRO)

     VALUES
           (origem.Id_SEGMENTO, 
            origem.Id_PAIS,
            origem.Id_PRODUTO,
            origem.Id.TIPO_DESCONTO,
            origem.QTDE_VENDA,
            origem.PRECO_CUSTO,
            origem.PECO_VENDA,
            origem.VENDA_BRUTA,
            origem.DESCONTO,
            origem.VENDAS,
            origem_CUSTO_VENDA,
            origem.LUCRO);

END TRY  
BEGIN CATCH
   IF @@TRANCOUNT > 0  
        ROLLBACK TRANSACTION; 

INSERT INTO [dbo].[LOG_CARGAS]
           ([NUMERO_ERRO],
           [SERVERIDADE_ERRO],
           [ESTADO_ERRO],
           [PROC_ERRO],
           [LINHA_ERRO],
           [MSG_ERRO],
           [SITUACAO],
           [PROCESSO],
           [DATA_INI],
         [DATA_FIM])
     VALUES
        (ERROR_NUMBER(),
        ERROR_SEVERITY(),
        ERROR_STATE(),
        ERROR_PROCEDURE(),
        ERROR_LINE() ,
        ERROR_MESSAGE() ,
        'ERRO',
        @VPROCESSO,
      @VDATA_INI,
      GETDATE()
      )

        
END CATCH;  

INSERT INTO LOG_CARGAS
           ([NUMERO_ERRO],
           [SERVERIDADE_ERRO],
           [ESTADO_ERRO],
           [PROC_ERRO],
           [LINHA_ERRO],
           [MSG_ERRO],
           [SITUACAO],
           [PROCESSO],
           [DATA_INI],
           [DATA_FIM])
     VALUES
        (ERROR_NUMBER(),
        ERROR_SEVERITY(),
        ERROR_STATE(),
        ERROR_PROCEDURE(),
        ERROR_LINE() ,
        ERROR_MESSAGE() ,
        'SUCESSO',
        @VPROCESSO,
      @VDATA_INI,
      GETDATE()
      )

IF @@TRANCOUNT > 0  
    COMMIT TRANSACTION;  
GO  

: Msg 208, Level 16, State 1, Line 9
Nome de objeto 'D_SEGMENTO' inválido.

Total execution time: 00:00:00.004

**Processo finalizado**